# Modelos Preditivos de Matemática Aplicada a Ciência de Dados

### Modelos Utilizados

## Classificação

- GaussianNB
- DecisionTreeClassifier
- KNeighborsClassifier
- RandomForestClassifier

## Regressão

- LinearRegression
- DecisionTreeRegressor
- KNeighborsRegressor
- RandomForestRegressor

### Link do dataset: https://www.kaggle.com/datasets/andriyaniarimbi/predictive-maintenance

# Imports

In [132]:
## Modelos

### Modelos de classificação
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

### Modelos de Regressão
import numpy as np
from sklearn.linear_model import LinearRegression


## Processamento e normalização de dados
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import StandardScaler

## Testes e Avaliação
from sklearn.model_selection import train_test_split, GridSearchCV

## Métricas
from sklearn.metrics import accuracy_score, r2_score,precision_score, recall_score, confusion_matrix, f1_score

from sklearn.base import is_classifier, is_regressor


## Processo de carregar e filtrar os dados do dataset

In [133]:
# Carregar o dataset em um dataframe
df = pd.read_csv('./datasets/predictive_maintenance.csv')

# Deletar colunas que não são uteis
df_processed = df.drop(['UDI', 'Product ID', 'Target'], axis=1)

# A coluna Failure Type é a coluna que vai ser preditada precisa ser convertida de texto para números

label_encoder = LabelEncoder()
df_processed['Failure Type'] = label_encoder.fit_transform(df_processed['Failure Type'])

# A coluna 'Type' é uma feature. Vamos usar a codificação one-hot para convertê-la
# Isso impede que o modelo pense que 'L' < 'M' < 'H'.
df_processed = pd.get_dummies(df_processed, columns=['Type'], drop_first=True)

# Separando as colunas preditada e o resto
X = df_processed.drop('Failure Type', axis=1)
y = df_processed['Failure Type']

print("\nProcessed Features (X) head:")
print(X.head())


Processed Features (X) head:
   Air temperature [K]  Process temperature [K]  Rotational speed [rpm]  \
0                298.1                    308.6                    1551   
1                298.2                    308.7                    1408   
2                298.1                    308.5                    1498   
3                298.2                    308.6                    1433   
4                298.2                    308.7                    1408   

   Torque [Nm]  Tool wear [min]  Type_L  Type_M  
0         42.8                0   False    True  
1         46.3                3    True   False  
2         49.4                5    True   False  
3         39.5                7    True   False  
4         40.0                9    True   False  


In [134]:
display(df_processed)

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,1,False,True
1,298.2,308.7,1408,46.3,3,1,True,False
2,298.1,308.5,1498,49.4,5,1,True,False
3,298.2,308.6,1433,39.5,7,1,True,False
4,298.2,308.7,1408,40.0,9,1,True,False
...,...,...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,1,False,True
9996,298.9,308.4,1632,31.8,17,1,False,False
9997,299.0,308.6,1645,33.4,22,1,False,True
9998,299.0,308.7,1408,48.5,25,1,False,False


In [135]:
df_processed['Failure Type'].unique()

array([1, 3, 5, 2, 4, 0])

In [136]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print(f"\nQuantidade de linhas de Treinamento: {x_train.shape}")
print(f"\nQuantidade de linhas de Teste: {x_test.shape}")


Quantidade de linhas de Treinamento: (8000, 7)

Quantidade de linhas de Teste: (2000, 7)


In [137]:

# Colunas que são numericas e precisam de normalização

numerical_features = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']

#Inicializar o scaler
scaler = StandardScaler()

# Colocar o scaler para aprender os parâmetros de normalização (média e desvio padrão) a partir dos dados de treino
scaler.fit(x_train[numerical_features])

# Transoformar os dados de treino e teste
x_train[numerical_features] = scaler.transform(x_train[numerical_features])
x_test[numerical_features] = scaler.transform(x_test[numerical_features])

print("\nX_train normalizado:")
print(x_train.head())


X_train normalizado:
      Air temperature [K]  Process temperature [K]  Rotational speed [rpm]  \
9254            -0.854066                -0.609589                0.427634   
1561            -0.904014                -1.080528               -0.834945   
1670            -0.904014                -1.484190               -0.059677   
6087             0.444571                 0.534121                0.333495   
6669             0.694309                 0.332290                0.178441   

      Torque [Nm]  Tool wear [min]  Type_L  Type_M  
9254    -0.892696         1.375035    True   False  
1561     1.382187         0.457620    True   False  
1670    -0.892696         1.359218    True   False  
6087    -0.702288        -1.598655   False    True  
6669    -0.612094         1.580663    True   False  


## Classificação do modelo


In [138]:
# --- Determinar automaticamente o tipo de problema ---
# Número de valores distintos na variável alvo
unique_values = y.nunique()
print(f"\nValores únicos na variável alvo: {unique_values}")
print(f"Exemplos de valores: {y.unique()[:10]}")

# Verifica o tipo de problema
if y.dtype == 'object' or unique_values < 20:
    problema = "classificacao"
else:
    problema = "regressao"

print(f"\nTipo de problema detectado: {problema.upper()}")



Valores únicos na variável alvo: 6
Exemplos de valores: [1 3 5 2 4 0]

Tipo de problema detectado: CLASSIFICACAO


# Modelos


In [139]:
if problema == "classificacao":
    models = {
        "GaussianNB": (
            GaussianNB(),
            {"var_smoothing": np.logspace(0, -9, num=100)}
        ),
        "DecisionTreeClassifier": (
            DecisionTreeClassifier(random_state=42),
            {
                "criterion": ["gini", "entropy"],
                "max_depth": [None, 10, 20, 30],
                "min_samples_split": [2, 5, 10],
                "min_samples_leaf": [1, 2, 4],
                "max_features": ["sqrt", "log2", None]
            }
        ),
        "KNNClassifier": (
            KNeighborsClassifier(),
            {
                "n_neighbors": [3, 5, 7, 9, 11],
                "weights": ["uniform", "distance"],
                "metric": ["euclidean", "manhattan"]
            }
        ),
        "RandomForestClassifier": (
            RandomForestClassifier(),
            {
                'n_estimators': [100, 200],
                'max_depth': [None, 10, 20],
                'min_samples_split': [2, 5],
                'min_samples_leaf': [1, 2],
                'max_features': ['sqrt', 'log2'],
                'criterion': ['gini', 'entropy'],
            }
        )
    }

else:  # problema == "regressao"
    models = {
        "LinearRegression": (
            LinearRegression(),
            {
                'fit_intercept': [True, False],
                'copy_X': [True, False]
            }
        ),
        "RandomForestRegressor": (
            RandomForestRegressor(),
            {
                'n_estimators': [100,200],
                'max_depth': [None, 10, 20],
                'min_samples_split': [2, 5],
                'min_samples_leaf': [1, 2],
                'max_features': ['sqrt', 'log2'],   # 'auto' = n_features
                'criterion': ['squared_error']  
            }
        ),
        "KNeighborsRegressor":(
            KNeighborsRegressor(),
            {
                'n_neighbors': [3,5,7,9,11],
                'weights': ['uniform','distance'],
                'algorithm': ['auto','ball_tree','kd_tree','brute'],
                'leaf_size': [20,30,40,50],
                'p': [1,2],
                'metric': ['minkowski','euclidean','manhattan']
            }
        ),
        "DecisionTreeRegressor":(
            DecisionTreeRegressor(),
            {
                "criterion": ["squared_error"],
                "max_depth": [None, 10, 20, 30],
                "min_samples_split": [2, 5, 10],
                "min_samples_leaf": [1, 2, 4],
                "max_features": ["sqrt", "log2", None]
            }
        )
    }


# Dicionários para armazenar resultados
best_models = {}
metrics = {}

# Loop para treinar e avaliar cada modelo
for name, (model, params) in models.items():
    print(f"\nTreinando {name}...")

    # Detecta automaticamente o tipo do modelo
    modelo_tipo = "regressor" if is_regressor(model) else "classificador"

    # Define a métrica de pontuação correta
    scoring_metric = 'r2' if modelo_tipo == "regressor" else 'accuracy'

    gs = GridSearchCV(
        estimator=model,
        param_grid=params,
        scoring=scoring_metric,
        cv=5,
        n_jobs=-1,
        verbose=1
    )
    gs.fit(x_train, y_train)

    print(f"Melhores parâmetros para {name}: {gs.best_params_}")

    best_model = gs.best_estimator_
    best_models[name] = best_model

    # Previsões no conjunto de teste
    y_pred = best_model.predict(x_test)

    # prepara dicionário com todas as chaves (garante consistência entre modelos)
    model_metrics = {
        "Accuracy": np.nan,
        "Precision": np.nan,
        "Recall": np.nan,
        "F1-Score": np.nan,
        "Confusion Matrix": None,
    }

    # Se for regressão, converte previsões contínuas para rótulos e calcula métricas de classificação + R²
    if modelo_tipo == "regressor":
        # arredonda para o inteiro mais próximo e limita ao intervalo das classes existentes
        y_pred_labels = np.rint(y_pred).astype(int)
        y_pred_labels = np.clip(y_pred_labels, int(y_test.min()), int(y_test.max()))

        # métricas de classificação usando as previsões convertidas
        acc = accuracy_score(y_test, y_pred_labels)
        prec = precision_score(y_test, y_pred_labels, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred_labels, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred_labels, average='weighted', zero_division=0)
        cm = confusion_matrix(y_test, y_pred_labels, labels=np.unique(y_test))

        model_metrics["Accuracy"] = acc
        model_metrics["Precision"] = prec
        model_metrics["Recall"] = rec
        model_metrics["F1-Score"] = f1
        model_metrics["Confusion Matrix"] = cm

    else:
        # Classificação padrão
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        cm = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
        
        model_metrics["Accuracy"] = acc
        model_metrics["Precision"] = prec
        model_metrics["Recall"] = rec
        model_metrics["F1-Score"] = f1
        model_metrics["Confusion Matrix"] = cm

    # salva métricas padronizadas
    metrics[name] = model_metrics

# Exibe resumo geral das métricas
summary_df = pd.DataFrame(metrics).T

# Remove a coluna da matriz de confusão apenas se ela existir
if "Confusion Matrix" in summary_df.columns:
    summary = summary_df.drop(columns="Confusion Matrix")
else:
    summary = summary_df



Treinando GaussianNB...


Fitting 5 folds for each of 100 candidates, totalling 500 fits
Melhores parâmetros para GaussianNB: {'var_smoothing': np.float64(1.0)}

Treinando DecisionTreeClassifier...
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Melhores parâmetros para DecisionTreeClassifier: {'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 2}

Treinando KNNClassifier...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Melhores parâmetros para KNNClassifier: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}

Treinando RandomForestClassifier...
Fitting 5 folds for each of 96 candidates, totalling 480 fits
Melhores parâmetros para RandomForestClassifier: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


# Métricas de Avaliação

In [140]:

print("\n\nResumo Final das Métricas:\n")
print(summary)



Resumo Final das Métricas:

                       Accuracy Precision  Recall  F1-Score
GaussianNB                0.969  0.945516   0.969  0.955933
DecisionTreeClassifier   0.9755  0.969122  0.9755  0.972152
KNNClassifier            0.9715   0.96424  0.9715  0.964424
RandomForestClassifier    0.981  0.972538   0.981  0.976133


##